In [1]:
!pip install gradio PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.6 MB/s eta 0:00:00


In [2]:
import gradio as gr
from PyPDF2 import PdfReader
import re
import random

def extract_text_from_pdf(pdf_path):
    text = ""
    try:
        reader = PdfReader(pdf_path)
        for page in reader.pages:
            text += page.extract_text() or ""
    except Exception as e:
        return f"Hata: PDF dosyası okunurken bir sorun oluştu: {e}"
    return text

def extract_value(text, key_patterns):
    for key_pattern in key_patterns:
        match = re.search(rf'{re.escape(key_pattern)}[^,\n]*?([0-9]+\.?[0-9]*|[0-9]*,[0-9]+)', text, re.IGNORECASE)
        if match:
            try:
                return float(match.group(1).replace(',', '.'))
            except ValueError:
                return None
    return None


bolge_options = [
    "Akdeniz", "Ege", "Marmara", "İç Anadolu", "Karadeniz", "Doğu Anadolu-Güneydoğu Anadolu"
]
gelir_options = [
    "0 TL - 30.000 TL",
    "30.001 TL - 60.000 TL",
    "60.001 TL - 110.000 TL",
    "110.001 TL ve üzeri"
]
yas_options = ["0-17", "18-30", "31-50", "51-65", "65+"]
yak_derece_options = [
    "1. Derece (Anne, Baba, Kardeş)",
    "2. Derece (Dede, Nine, Amca, Teyze)",
    "3. Derece (Kuzen, Yeğen)",
    "4. Derece (Uzak Akraba)"
]
kalitsal_hastalik_tip_options = [
    "1. Anemi (Hemoglobin ile ilgili)",
    "2. Osteoporoz (Magnezyum ile ilgili)",
    "3. Diyabet (Glukoz ile ilgili)",
    "4. Böbrek Hastalığı (Kreatinin ile ilgili)",
    "5. Karaciğer Hastalığı (Üre ile ilgili)",
    "6. Raşitizm (D Vitamini ile ilgili)",
    "7. Hipertansiyon (Potasyum ile ilgili)"
]


NORMAL_COMMENTS = [
    "Seviyeniz normal aralıkta, bu çok iyi bir durum.",
    "Değeriniz ideal sınırlarda seyrediyor, tebrikler!",
    "Her şey yolunda, sonuçlarınız sağlıklı bir tablo çiziyor.",
    "Referans aralığında olması oldukça olumlu bir işaret.",
    "Vücudunuzun bu alanda iyi çalıştığını gösteriyor."
]


LOW_COMMENTS = [
    "Seviyeniz düşük, bu durum potansiyel bir sorun olabileceğini düşündürüyor. Lütfen doktorunuza danışın.",
    "Değeriniz referans aralığının altında. Daha detaylı bir inceleme gerekebilir.",
    "Düşük sonuç, bazı belirtilere yol açabilir. Bir sağlık uzmanıyla görüşmeniz faydalı olacaktır.",
    "Beklenenin altında bir değer, bu durumun nedenleri araştırılmalı.",
    "Bu düşük sonuç, genel sağlığınız için önemli bir gösterge olabilir. Tıbbi yardım almayı düşünün."
]


HIGH_COMMENTS = [
    "Seviyeniz yüksek, bu durumun nedenleri hakkında bir uzmana danışmalısınız.",
    "Değeriniz referans aralığının üzerinde. Sağlığınız için bir uyarı olabilir.",
    "Yüksek sonuç, olası bir rahatsızlığa işaret edebilir. Bir sağlık profesyoneli ile konuşmanız önemlidir.",
    "Beklenenin üzerinde bir değer, bu durumun nedenleri araştırılmalı.",
    "Bu yüksek sonuç, genel sağlığınız için önemli bir gösterge olabilir. Tıbbi yardım almayı düşünün."
]


def get_hemoglobin_comments(hemoglobin, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir):
    comments = []
    if hemoglobin is None:
        return ["Hemoglobin değeri okunamadı."]
    if 12.6 <= hemoglobin <= 16.7:
        comments.append(random.choice(NORMAL_COMMENTS))
    elif hemoglobin < 12.6:
        comments.append(random.choice(LOW_COMMENTS))
        if has_hereditary and "1. Anemi (Hemoglobin ile ilgili)" in kalitsal_hastalik_tip:
            if yak_derece == "1. Derece (Anne, Baba, Kardeş)":
                comments.append(random.choice([
                    "Hemoglobin değeriniz referans aralığının dışında seyrediyor. Ailenizde 1. düzeyde bu tür bir rahatsızlık geçmişi olduğundan derhal bir uzmana başvurmanız kritik önem taşıyor.",
                    "Kanınızdaki hemoglobin seviyesi normal sınırların dışında. 1. düzey akrabalarınızda genetik yatkınlık göz önüne alındığında, vakit kaybetmeden doktora görünmeniz önerilir.",
                    "Hemoglobin değeriniz istenilen seviyede değil. 1. derece akrabalarınızda benzer bir kalıtsal durum olması nedeniyle acil tıbbi değerlendirme gereklidir.",
                    "Acil durum! Hemoglobin değeriniz referans dışı ve aile geçmişinizde ciddi bir yatkınlık var. Hemen bir sağlık profesyoneline başvurun.",
                    "Çok önemli! Kanınızdaki hemoglobin seviyesi alarm veriyor ve 1. derece akrabanızda benzer bir durum var. En kısa sürede doktorunuza gidin."
                ]))
            else:
                comments.append(random.choice([
                    "Ailenizde anemi öyküsü mevcut. Bu durumun genetik bağlantısı için doktorunuza danışın.",
                    "Genetik yatkınlık nedeniyle anemi riskiniz olabilir. Bir uzmana görünmeniz faydalı olacaktır.",
                    "Akrabalarınızda anemi bulunması, sizin de bu durumu yaşama ihtimalinizi artırıyor. Kontrol altında tutulmalı.",
                    "Ailenizde görülen anemi, sizin için de bir risk faktörü. Lütfen doktorunuzla konuşun.",
                    "Anemi aile geçmişinizde yer alıyor. Bu nedenle, kendi değerlerinizi düzenli olarak kontrol ettirin."
                ]))

        if yas == "0-17":
            if "0 TL - 30.000 TL" in gelir:
                comments.append(random.choice([
                    "Çocuk ve ergenlerde düşük gelir düzeyine sahip ailelerde demir eksikliği anemisi riski artabilir. Beslenme düzenine dikkat edilmeli.",
                    "Düşük gelirli ailelerdeki çocuklarda anemi görülme olasılığı daha yüksek. Beslenme danışmanlığı faydalı olabilir.",
                    "Yetersiz beslenme, özellikle düşük gelirli genç bireylerde anemiyi tetikleyebilir. Tıbbi yardım alınmalı.",
                    "Çocukluk çağındaki anemi, gelir düzeyine bağlı olarak beslenme yetersizliklerinden kaynaklanabilir. Bir sağlık kuruluşuna başvurun.",
                    "Bu yaş grubunda ve düşük gelirli çocuklarda anemi riski artış gösteriyor. Lütfen beslenme alışkanlıklarını gözden geçirin."
                ]))
            else:
                comments.append(random.choice([
                    "Çocuk ve ergenlerde düşük hemoglobin, büyüme ve gelişim açısından önem taşır. Bir pediatr ile görüşmelisiniz.",
                    "Bu yaş grubunda düşük hemoglobin, beslenme veya başka bir sağlık sorununa işaret edebilir. Doktor kontrolü önerilir.",
                    "Çocukluk çağındaki anemi, okul başarısını ve genel aktivite düzeyini etkileyebilir. Nedenleri araştırılmalı.",
                    "Genç yaşta görülen düşük hemoglobin, demir eksikliği başta olmak üzere çeşitli nedenlerden kaynaklanabilir. Uzman desteği alın.",
                    "Çocuklarda düşük hemoglobin seviyeleri, dikkatli bir takip ve değerlendirme gerektirir. Sağlık uzmanınıza danışın."
                ]))
        elif yas in ["18-30", "31-50"]:
            if "0 TL - 30.000 TL" in gelir:
                comments.append(random.choice([
                    f"{yas} yaş grubunda {gelir} düzeyine sahip kişilerde demir eksikliği anemisi riski artabilir. Beslenmenizi gözden geçirin.",
                    f"{gelir} gelirli genç yetişkinlerde anemi yaygın bir durum olabilir. Beslenme düzeninize dikkat etmeniz önemlidir.",
                    f"{yas} aralığında ve düşük gelir düzeyinde anemi, yetersiz beslenme veya kronik hastalıklarla ilişkili olabilir. Tıbbi destek alın.",
                    "Çalışma çağındaki bireylerde düşük hemoglobin, verimliliği etkileyebilir. Sosyoekonomik faktörler de etkili olabilir. Doktorunuza danışın.",
                    f"{gelir} genç ve orta yaşlı bireylerde anemi riski yüksek. Beslenme alışkanlıklarınızı gözden geçirin ve tıbbi yardım alın."
                ]))
            else:
                comments.append(random.choice([
                    "Genç ve orta yaş grubunda düşük hemoglobin, demir eksikliği, kan kaybı veya kronik hastalıklarla ilişkili olabilir. Nedenleri araştırılmalı.",
                    f"{yas} aralığında düşük hemoglobin seviyeniz varsa, detaylı bir değerlendirme için doktorunuza başvurun.",
                    "Düşük hemoglobin, yorgunluk ve enerji düşüklüğüne yol açabilir. Beslenme alışkanlıklarınızı gözden geçirin.",
                    "Genç yetişkinlerde anemi, çeşitli tıbbi durumların bir belirtisi olabilir. Uzman görüşü almak önemlidir.",
                    "Hemoglobininiz düşük, bu durum yaşam kalitenizi etkileyebilir. Bir hekime danışarak çözüm yolları arayın."
                ]))
        elif yas in ["51-65", "65+"]:
            comments.append(random.choice([
                f"{yas} aralığında düşük hemoglobin, kronik hastalıklar veya beslenme yetersizlikleriyle ilişkili olabilir. Detaylı tıbbi değerlendirme önemlidir.",
                f"{yas} aralığında düşük hemoglobin, altta yatan başka bir sağlık sorununun işareti olabilir. Doktorunuza danışın.",
                "Düşük hemoglobin seviyesi, yaşlılarda genel zayıflık ve yorgunluğa neden olabilir. Beslenmenizi ve ilaçlarınızı gözden geçirin.",
                "Yaş ilerledikçe hemoglobin düşüşü görülebilir, ancak nedenini araştırmak için bir hekime danışmalısınız.",
                f"Kronik hastalıklar, {yas} aralığında düşük hemoglobine yol açabilir. Tıbbi takibiniz önemlidir."
            ]))
    else:
        comments.append(random.choice(HIGH_COMMENTS))
        if has_hereditary and "1. Anemi (Hemoglobin ile ilgili)" in kalitsal_hastalik_tip:
            comments.append(random.choice([
                "Nadiren de olsa, bazı genetik faktörler yüksek hemoglobini etkileyebilir. Doktorunuza danışın.",
                "Kalıtsal yatkınlık durumunda yüksek hemoglobin değerleri, genetik bir durumu işaret edebilir. Bir genetik uzmanına danışın.",
                "Ailenizde görülen genetik hastalıklar, yüksek hemoglobin seviyenizle ilişkili olabilir. Tıbbi geçmişinizi paylaşın.",
                "Yüksek hemoglobinin genetik nedenleri olabilir. Bu durumu doktorunuzla detaylıca konuşun.",
                "Genetik faktörler yüksek hemoglobini etkiliyorsa, özel bir tedavi planı gerekebilir."
            ]))
    return " ".join(comments)

def get_magnezyum_comments(magnezyum, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir):
    comments = []
    if magnezyum is None:
        return ["Magnezyum değeri okunamadı."]
    if 1.6 <= magnezyum <= 2.6:
        comments.append(random.choice(NORMAL_COMMENTS))
    elif magnezyum < 1.6:
        comments.append(random.choice(LOW_COMMENTS))
        if has_hereditary and "2. Osteoporoz (Magnezyum ile ilgili)" in kalitsal_hastalik_tip:
            if yak_derece == "1. Derece (Anne, Baba, Kardeş)":
                comments.append(random.choice([
                    "Magnezyum değeriniz referans aralığının altında. Ailenizde 1. düzeyde osteoporoz öyküsü olduğundan kemik sağlığınız için acil tıbbi değerlendirme gereklidir.",
                    "Kanınızdaki magnezyum seviyesi düşük. 1. düzey akrabalarınızda genetik yatkınlık göz önüne alındığında, doktora başvurmanız önemlidir.",
                    "Magnezyum seviyeniz düşük ve ailenizde osteoporoz öyküsü olduğu için, kemik sağlığınız için acil tıbbi değerlendirme gereklidir.",
                    "Kanınızdaki magnezyum düzeyi referans aralığının altında. Birinci derece akrabalarınızdaki genetik yatkınlık göz önüne alındığında, doktora başvurmanız önemlidir.",
                    "Düşük magnezyum değeriniz ve ailenizdeki osteoporoz geçmişi nedeniyle, kemik sağlığınızı korumak için tıbbi yardım almak kritik bir adımdır."
                ]))
            else:
                comments.append(random.choice([
                    "Ailenizde osteoporoz öyküsü mevcut. Magnezyum takviyesi veya beslenme düzenlemesi gerekebilir.",
                    "Genetik yatkınlık nedeniyle osteoporoz riskiniz olabilir. Magnezyum seviyeniz düşükse, bu riski artırabilir. Bir uzmana görünmeniz faydalı olacaktır.",
                    "Akrabalarınızda osteoporoz bulunması, sizin de bu durumu yaşama ihtimalinizi artırıyor. Magnezyum seviyenizi kontrol altında tutun.",
                    "Osteoporoz aile geçmişinizde yer alıyor. Bu nedenle, magnezyum takviyesi veya diyet değişiklikleri konusunda doktorunuzla konuşun.",
                    "Ailenizde görülen osteoporoz, sizin için de bir risk faktörü. Düşük magnezyumunuz bu riski artırıyor olabilir. Sağlık uzmanınıza danışın."
                ]))

        if yas in ["51-65", "65+"]:
            comments.append(random.choice([
                "İleri yaşlarda magnezyum eksikliği kemik yoğunluğu ve kas fonksiyonları için risk oluşturabilir. Takviye veya diyet düzenlemesi düşünülebilir.",
                f"{yas} aralığında magnezyum eksikliği, osteoporoz riskini artırabilir. Doktorunuzla konuşun.",
                f"Düşük magnezyum seviyesi, {yas} aralığında kas kramplarına ve genel halsizliğe neden olabilir. Beslenmenizi gözden geçirin.",
                f"{yas} aralığında magnezyum emilimi azalabilir. Bu durumu doktorunuzla paylaşın.",
                f"Magnezyum eksikliği, {yas} aralığında kemik erimesi ve düşme riskini artırabilir. Bir sağlık uzmanına danışın."
            ]))
        if "0 TL - 30.000 TL" in gelir:
            comments.append(random.choice([
                f"{gelir} düzeyine sahip kişilerde dengesiz beslenme nedeniyle magnezyum eksikliği görülebilir. Beslenme danışmanlığı faydalı olabilir.",
                f"Magnezyum açısından zengin gıdalara erişim kısıtlı olduğunda, {gelir} düzeyine sahip bireylerde eksiklik riski artar. Doktorunuza danışın.",
                f"Yetersiz beslenme, magnezyum eksikliğine yol açabilir. {gelir} düzeyine sahip olduğunuz için bu duruma dikkat edin.",
                f"Magnezyum eksikliği riski, {gelir} düzeyine sahip topluluklarda daha yüksek olabilir. Beslenme alışkanlıklarınızı gözden geçirin.",
                f"{gelir} düzeyine sahip bireylerde magnezyum eksikliği, beslenme yetersizliklerinden kaynaklanabilir. Tıbbi yardım alın."
            ]))
    else:
        comments.append(random.choice(HIGH_COMMENTS))
        if has_hereditary and "4. Böbrek Hastalığı (Kreatinin ile ilgili)" in kalitsal_hastalik_tip:
            comments.append(random.choice([
                "Ailenizde böbrek hastalığı öyküsü varsa, yüksek magnezyum böbrek fonksiyonları açısından değerlendirilmelidir.",
                "Kalıtsal yatkınlık durumunda yüksek magnezyum değerleri, böbrek fonksiyonlarında bir sorun olduğunu düşündürebilir. Bir uzmana danışın.",
                "Akrabalarınızda böbrek hastalığı bulunması, yüksek magnezyum seviyenizle ilişkili olabilir. Tıbbi geçmişinizi paylaşın.",
                "Yüksek magnezyumun böbrek hastalıklarıyla genetik bağlantısı olabilir. Bu durumu doktorunuzla detaylıca konuşun.",
                "Genetik faktörler yüksek magnezyumu etkiliyorsa, böbrek sağlığı için özel bir tedavi planı gerekebilir."
            ]))
    return " ".join(comments)

def get_glukoz_comments(glukoz, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir):
    comments = []
    if glukoz is None:
        return ["Glukoz değeri okunamadı."]
    if 74 <= glukoz <= 106:
        comments.append(random.choice(NORMAL_COMMENTS))
    elif glukoz > 106:
        comments.append(random.choice(HIGH_COMMENTS))
        if has_hereditary and "3. Diyabet (Glukoz ile ilgili)" in kalitsal_hastalik_tip:
            if yak_derece in ["1. Derece (Anne, Baba, Kardeş)", "2. Derece (Dede, Nine, Amca, Teyze)"]:
                comments.append(random.choice([
                    "Açlık kan şekeriniz yüksek. Ailenizde diyabet öyküsü olduğundan, yaşam tarzı değişiklikleri ve doktor kontrolü kritik önem taşıyor.",
                    "Kan şekeriniz normal sınırların dışında. Genetik yatkınlık göz önüne alındığında, vakit kaybetmeden doktora görünmeniz önerilir.",
                    "Açlık kan şekeriniz yüksek ve ailenizde diyabet öyküsü olduğu için, yaşam tarzı değişiklikleri yapmanız ve doktor kontrolünde olmanız hayati önem taşıyor.",
                    "Kan şekeriniz normal sınırların dışında. Ailenizdeki genetik yatkınlık göz önüne alındığında, vakit kaybetmeden bir doktora görünmeniz şiddetle tavsiye edilir.",
                    "Yüksek açlık kan şekeriniz ve ailenizdeki diyabet geçmişi nedeniyle, sağlıklı yaşam tarzı alışkanlıkları benimsemek ve tıbbi yardım almak kritik bir adımdır."
                ]))
            else:
                comments.append(random.choice([
                    "Ailenizde diyabet öyküsü mevcut. Kan şekeri takibiniz için doktorunuza danışın.",
                    "Genetik yatkınlık nedeniyle diyabet riskiniz olabilir. Kan şekeri seviyeniz yüksekse, bu riski artırabilir. Bir uzmana görünmeniz faydalı olacaktır.",
                    "Akrabalarınızda diyabet bulunması, sizin de bu durumu yaşama ihtimalinizi artırıyor. Kan şekeri seviyenizi kontrol altında tutun.",
                    "Diyabet aile geçmişinizde yer alıyor. Bu nedenle, kan şekeri takibi ve yaşam tarzı değişiklikleri konusunda doktorunuzla konuşun.",
                    "Ailenizde görülen diyabet, sizin için de bir risk faktörü. Yüksek glukozunuz bu riski artırıyor olabilir. Sağlık uzmanınıza danışın."
                ]))

        if yas in ["51-65", "65+"]:
            comments.append(random.choice([
                f"{yas} düzeyinde diyabet riski artmaktadır. Kan şekerinizi düzenli kontrol ettirmeniz ve yaşam tarzınıza dikkat etmeniz önemlidir.",
                f"{yas} düzeyinde diyabet görülme sıklığı artmaktadır. Kan şekeri takibinizi aksatmayın.",
                f"Yüksek kan şekeri, {yas} düzeyinde ciddi sağlık sorunlarına yol açabilir. Doktorunuzla görüşün.",
                "Yaş ilerledikçe, diyabet yönetimi daha kritik hale gelir. Kan şekerinizi düzenli ölçtürün.",
                "Diyabet riski yaşla birlikte artar. Sağlıklı yaşam tarzı ve düzenli doktor kontrolü önemlidir."
            ]))
        if "0 TL - 30.000 TL" in gelir:
            comments.append(random.choice([
                f"{gelir} düzeyine sahip kişilerde yetersiz beslenme veya sağlıksız beslenme alışkanlıkları kan şekeri regülasyonunu etkileyebilir. Beslenme danışmanlığı önemlidir.",
                f"Sağlıksız gıdalara erişim, {gelir} düzeyine sahip bireylerde diyabet riskini artırabilir. Doktorunuzla konuşun.",
                f"{gelir} düzeyine sahipseniz, kan şekeri kontrolünüz için beslenme alışkanlıklarınızı gözden geçirin.",
                f"Diyabet riski, {gelir} düzeyine sahip topluluklarda beslenme alışkanlıkları nedeniyle daha yüksek olabilir. Bir uzmana danışın.",
                f"Beslenme yetersizlikleri, {gelir} düzeyine sahip bireylerde kan şekeri düzensizliklerine yol açabilir. Tıbbi yardım alın."
            ]))
    else:
        comments.append(random.choice(LOW_COMMENTS))
    return " ".join(comments)

def get_ure_comments(ure, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir):
    comments = []
    if ure is None:
        return ["Üre değeri okunamadı."]
    if 0 <= ure <= 50:
        comments.append(random.choice(NORMAL_COMMENTS))
    elif ure > 50:
        comments.append(random.choice(HIGH_COMMENTS))
        if has_hereditary and "4. Böbrek Hastalığı (Kreatinin ile ilgili)" in kalitsal_hastalik_tip:
            if yak_derece == "1. Derece (Anne, Baba, Kardeş)":
                comments.append(random.choice([
                    "Üre değeriniz yüksek. Ailenizde 1. düzeyde böbrek hastalığı öyküsü olduğundan detaylı bir böbrek fonksiyon testi yaptırmanız acil önem taşıyor.",
                    "Kanınızdaki üre seviyesi normal sınırların dışında. 1. düzey akrabalarınızda genetik yatkınlık göz önüne alındığında, vakit kaybetmeden doktora görünmeniz önerilir.",
                    "Üre seviyeniz yüksek ve ailenizde böbrek hastalığı öyküsü olduğu için, detaylı bir böbrek fonksiyon testi yaptırmanız acil önem taşıyor.",
                    "Kanınızdaki üre düzeyi normalin dışında. Birinci derece akrabalarınızdaki genetik yatkınlık göz önüne alındığında, vakit kaybetmeden bir doktora görünmeniz önerilir.",
                    "Yüksek üre değeriniz ve ailenizdeki böbrek rahatsızlığı geçmişi nedeniyle, acil tıbbi değerlendirme ve böbrek fonksiyon testi yaptırmanız gereklidir."
                ]))
            else:
                comments.append(random.choice([
                    "Ailenizde böbrek hastalığı öyküsü mevcut. Böbrek sağlığınız için doktorunuza danışın.",
                    "Genetik yatkınlık nedeniyle böbrek hastalığı riskiniz olabilir. Üre seviyeniz yüksekse, bu riski artırabilir. Bir uzmana görünmeniz faydalı olacaktır.",
                    "Akrabalarınızda böbrek hastalığı bulunması, sizin de bu durumu yaşama ihtimalinizi artırıyor. Üre seviyenizi kontrol altında tutun.",
                    "Böbrek hastalığı aile geçmişinizde yer alıyor. Bu nedenle, üre takibi ve yaşam tarzı değişiklikleri konusunda doktorunuzla konuşun.",
                    "Ailenizde görülen böbrek hastalığı, sizin için de bir risk faktörü. Yüksek üreniz bu riski artırıyor olabilir. Sağlık uzmanınıza danışın."
                ]))

        if yas in ["51-65", "65+"]:
            comments.append(random.choice([
                f"{yas} yaşlarda böbrek fonksiyonları azalabileceğinden, yüksek üre dikkat gerektirir. Detaylı böbrek fonksiyon testleri önerilir.",
                "Yaşlılarda yüksek üre, böbrek yetmezliği belirtisi olabilir. Doktorunuza danışın.",
                f"Yüksek üre, {yas} grubunda böbrek sağlığı açısından önemlidir. Düzenli takip gereklidir.",
                f"Böbrekler, {yas} grubunda daha az etkili çalışabilir. Yüksek üre bir uyarı olabilir, hekime başvurun.",
                f"Kronik böbrek hastalıkları, {yaş} grubunda yüksek üreye yol açabilir. Tıbbi takibiniz önemlidir."
            ]))
    else:
        comments.append(random.choice(LOW_COMMENTS))
    return " ".join(comments)

def get_kreatinin_comments(kreatinin, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir):
    comments = []
    if kreatinin is None:
        return ["Kreatinin değeri okunamadı."]
    if 0.67 <= kreatinin <= 1.17:
        comments.append(random.choice(NORMAL_COMMENTS))
    elif kreatinin > 1.17:
        comments.append(random.choice(HIGH_COMMENTS))
        if has_hereditary and "4. Böbrek Hastalığı (Kreatinin ile ilgili)" in kalitsal_hastalik_tip:
            if yak_derece == "1. Derece (Anne, Baba, Kardeş)":
                comments.append(random.choice([
                    "Kreatinin değeriniz yüksek. Ailenizde 1. düzeyde böbrek hastalığı öyküsü olduğundan böbrek yetmezliği riski taşıyabilir. Acil tıbbi değerlendirme önerilir.",
                    "Kanınızdaki kreatinin seviyesi normal sınırların dışında. 1. düzey akrabalarınızda genetik yatkınlık göz önüne alındığında, vakit kaybetmeden doktora görünmeniz önemlidir."
                    "Kreatinin seviyeniz yüksek ve ailenizde böbrek hastalığı öyküsü olduğu için acil bir doktora görünmeniz şiddetle tavsiye edilir.",
                    "Kanınızdaki yüksek kreatinin düzeyi ve genetik yatkınlığınız nedeniyle böbrek yetmezliği riski taşıyorsunuz, bu yüzden hemen tıbbi yardım almalısınız.",
                    "Kreatinin değerinizin normalin üzerinde olması ve ailenizdeki böbrek rahatsızlığı geçmişi göz önüne alındığında, gecikmeden bir uzmana danışmanız hayati önem taşımaktadır."
                ]))
            else:
                comments.append(random.choice([
                    "Ailenizde böbrek hastalığı öyküsü mevcut. Böbrek fonksiyonlarınızı kontrol ettirin.",
                    "Genetik yatkınlık nedeniyle böbrek hastalığı riskiniz olabilir. Kreatinin seviyeniz yüksekse, bu riski artırabilir. Bir uzmana görünmeniz faydalı olacaktır.",
                    "Akrabalarınızda böbrek hastalığı bulunması, sizin de bu durumu yaşama ihtimalinizi artırıyor. Kreatinin seviyenizi kontrol altında tutun.",
                    "Böbrek hastalığı aile geçmişinizde yer alıyor. Bu nedenle, kreatinin takibi ve yaşam tarzı değişiklikleri konusunda doktorunuzla konuşun.",
                    "Ailenizde görülen böbrek hastalığı, sizin için de bir risk faktörü. Yüksek kreatinininiz bu riski artırıyor olabilir. Sağlık uzmanınıza danışın."
                ]))

        if yas in ["51-65", "65+"]:
            comments.append(random.choice([
                f"{yas} yaşlarda yüksek kreatinin böbrek sağlığı açısından önemlidir. Düzenli takip gereklidir.",
                f"{yas} grubunda yüksek kreatinin, böbrek yetmezliğinin bir göstergesi olabilir. Doktorunuza danışın.",
                f"Yüksek kreatinin seviyesi, {yas} grubunda böbrek fonksiyonlarının azalmasına işaret edebilir. Bir hekime başvurun.",
                "Böbrekler, yaşlandıkça daha az etkili çalışabilir. Yüksek kreatinin, böbrek sağlığınız için önemli bir uyarıdır.",
                f"Kronik böbrek hastalıkları, {yas} grubunda yüksek kreatinine yol açabilir. Tıbbi takibiniz önemlidir."
            ]))
    else:
        comments.append(random.choice(LOW_COMMENTS))
    return " ".join(comments)

def get_dvitamini_comments(d_vitamini, has_hereditary, yak_derece, kalitsal_hastalik_tip, gelir, bolge):
    comments = []
    if d_vitamini is None:
        return ["D Vitamini değeri okunamadı."]
    if 30 <= d_vitamini <= 100:
        comments.append(random.choice(NORMAL_COMMENTS))
    elif d_vitamini < 30:
        comments.append(random.choice(LOW_COMMENTS))
        if has_hereditary and "6. Raşitizm (D Vitamini ile ilgili)" in kalitsal_hastalik_tip:
            if yak_derece in ["1. Derece (Anne, Baba, Kardeş)", "2. Derece (Dede, Nine, Amca, Teyze)"]:
                comments.append(random.choice([
                    "D Vitamini değeriniz düşük. Ailenizde raşitizm öyküsü olduğundan, genetik yatkınlık düşündürebilir. Takviye ve doktor kontrolü gereklidir.",
                    "Kanınızdaki D Vitamini seviyesi normal sınırların dışında. Genetik yatkınlık göz önüne alındığında, vakit kaybetmeden doktora görünmeniz önerilir.",
                    "D Vitamini seviyeniz düşük ve ailenizde raşitizm öyküsü bulunduğundan, takviye almanız ve doktor kontrolünden geçmeniz önemlidir.",
                    "Kanınızdaki D Vitamini düzeyinin normalin altında olması ve genetik yatkınlığınız nedeniyle, bir an önce doktora görünmeniz faydalı olacaktır.",
                    "D Vitamini değerinizin yetersiz olduğu ve ailenizde raşitizm geçmişi olduğu için, tıbbi değerlendirme ve takviye gereklidir."
                ]))
            else:
                comments.append(random.choice([
                    "Ailenizde raşitizm öyküsü mevcut. D vitamini eksikliği riskiniz artmış olabilir.",
                    "Genetik yatkınlık nedeniyle raşitizm riskiniz olabilir. D vitamini seviyeniz düşükse, bu riski artırabilir. Bir uzmana görünmeniz faydalı olacaktır.",
                    "Akrabalarınızda raşitizm bulunması, sizin de bu durumu yaşama ihtimalinizi artırıyor. D vitamini seviyenizi kontrol altında tutun.",
                    "Raşitizm aile geçmişinizde yer alıyor. Bu nedenle, D vitamini takviyesi veya diyet değişiklikleri konusunda doktorunuzla konuşun.",
                    "Ailenizde görülen raşitizm, sizin için de bir risk faktörü. Düşük D vitamininiz bu riski artırıyor olabilir. Sağlık uzmanınıza danışın."
                ]))

        if bolge in ["Karadeniz", "Doğu Anadolu-Güneydoğu Anadolu"]:
            comments.append(random.choice([
                f"Yaşadığınız bölge ({bolge}) kış aylarında veya genel olarak güneş ışığına az maruz kalmaya neden olabilir. D vitamini takviyesi veya güneşlenme önerilebilir.",
                f"{bolge} bölgesinde güneşlenme süresi daha kısa olabilir. D vitamini takviyesi almayı veya beslenme düzeninizi gözden geçirmeyi düşünebilirsiniz.",
                f"Güneş ışınlarının az olduğu {bolge} bölgesinde D vitamini eksikliği riski artabilir. Doktorunuzla bu konuyu görüşün.",
                f"{bolge} bölgesinde yeterli güneşlenememe, D vitamini eksikliğinin yaygın nedenlerinden biridir. Açık havada daha fazla vakit geçirmeyi deneyin.",
                f"Yaşadığınız {bolge} bölgesinin iklim koşulları D vitamini sentezinizi etkileyebilir. Takviye seçeneklerini doktorunuzla değerlendirin."
            ]))
        elif bolge in ["Marmara", "İç Anadolu"]:
            comments.append(random.choice([
                f"Yaşadığınız bölge ({bolge}) D vitamini sentezi için orta düzeyde güneş ışığı almaktadır. Yeterli güneşlenme ve beslenmeye dikkat edin.",
                f"{bolge} bölgesinde D vitamini seviyenizi korumak için yeterli güneşlenmeye ve D vitamini içeren besinlere odaklanın.",
                f"{bolge} bölgesinde yaşıyor olsanız da, D vitamini seviyenizi iyileştirmek için beslenme ve güneşlenme alışkanlıklarınızı gözden geçirin.",
                f"Orta derecede güneş alan {bolge} bölgesinde D vitamini seviyenizi korumak için bilinçli olmalısınız.",
                f"{bolge} bölgesindeki güneşlenme koşulları, D vitamini ihtiyacınızı kısmen karşılayabilir. Doktorunuzla ek takviye ihtiyacınızı görüşün."
            ]))
        elif bolge in ["Akdeniz", "Ege"]:
            comments.append(random.choice([
                f"Yaşadığınız bölge ({bolge}) bol güneş almasına rağmen D vitamini eksikliği görülmesi, yaşam tarzı veya diğer faktörlerle ilişkili olabilir. Doktorunuza danışın.",
                f"Güneşli bir bölge olan {bolge}'de D vitamini eksikliği, yaşam tarzı, güneş koruyucu kullanımı veya emilim sorunlarından kaynaklanabilir. Detaylı inceleme gerekebilir.",
                f"{bolge} bölgesinde yaşamanıza rağmen D vitamini eksikliği yaşıyorsanız, bu durum altta yatan başka bir sağlık sorununa işaret edebilir. Bir uzmana başvurun.",
                f"Bol güneşe rağmen D vitamini seviyenizin düşük olması şaşırtıcı olabilir. Doktorunuzla nedenlerini araştırmalısınız.",
                f"{bolge} bölgesinde olsanız bile, D vitamini eksikliği hala görülebilir. Düzenli kontroller ve doktor tavsiyesi önemlidir."
            ]))

        if "0 TL - 30.000 TL" in gelir:
            comments.append(random.choice([
                f"{gelir} düzeyine sahip kişilerde D vitamini açısından zengin gıdalara erişim kısıtlı olabilir. Takviye veya beslenme danışmanlığı düşünülebilir.",
                f"D vitamini eksikliği riski, {gelir} düzeyindeki bireylerde beslenme yetersizlikleri nedeniyle artabilir. Doktorunuzla konuşun.",
                f"Yetersiz beslenme, D vitamini eksikliğine yol açabilir. {gelir} düzeyine sahipseniz bu duruma dikkat edin.",
                f"D vitamini eksikliği riski, {gelir} düzeyineki topluluklarda daha yüksek olabilir. Beslenme alışkanlıklarınızı gözden geçirin.",
                f"{gelir} bireylerde D vitamini eksikliği, beslenme yetersizliklerinden kaynaklanabilir. Tıbbi yardım alın."
            ]))
    else:
        comments.append(random.choice(HIGH_COMMENTS))
    return " ".join(comments)

def get_potasyum_comments(potasyum, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir):
    comments = []
    if potasyum is None:
        return ["Potasyum değeri okunamadı."]
    if 3.9 <= potasyum <= 5.1:
        comments.append(random.choice(NORMAL_COMMENTS))
    elif potasyum < 3.9:
        comments.append(random.choice(LOW_COMMENTS))
        if has_hereditary and "7. Hipertansiyon (Potasyum ile ilgili)" in kalitsal_hastalik_tip:
            if yak_derece == "1. Derece (Anne, Baba, Kardeş)":
                comments.append(random.choice([
                    "Potasyum değeriniz düşük. Ailenizde hipertansiyon öyküsü olduğundan, tansiyon kontrolü için önemlidir. Diyetinize potasyum eklemeyi düşünebilirsiniz.",
                    "Kanınızdaki potasyum seviyesi normal sınırların dışında. Genetik yatkınlık göz önüne alındığında, vakit kaybetmeden doktora görünmeniz önerilir.",
                    "Potasyum seviyeniz düşük ve ailenizde hipertansiyon öyküsü olduğu için, tansiyon kontrolünüz açısından önemlidir. Diyetinize potasyum eklemeyi düşünebilirsiniz.",
                    "Kanınızdaki potasyum düzeyi normalin altında. Ailenizdeki genetik yatkınlık göz önüne alındığında, bir doktora görünmeniz faydalı olacaktır.",
                    "Potasyum değerinizin yetersiz olması ve ailenizde yüksek tansiyon geçmişi bulunması nedeniyle, tansiyonunuzu kontrol altında tutmak için tıbbi değerlendirme önemlidir."
                ]))
            else:
                comments.append(random.choice([
                    "Ailenizde hipertansiyon öyküsü mevcut. Potasyum seviyenizin takibi önemlidir.",
                    "Genetik yatkınlık nedeniyle hipertansiyon riskiniz olabilir. Potasyum seviyeniz düşükse, bu riski artırabilir. Bir uzmana görünmeniz faydalı olacaktır.",
                    "Akrabalarınızda hipertansiyon bulunması, sizin de bu durumu yaşama ihtimalinizi artırıyor. Potasyum seviyenizi kontrol altında tutun.",
                    "Hipertansiyon aile geçmişinizde yer alıyor. Bu nedenle, potasyum takviyesi veya diyet değişiklikleri konusunda doktorunuzla konuşun.",
                    "Ailenizde görülen hipertansiyon, sizin için de bir risk faktörü. Düşük potasyumunuz bu riski artırıyor olabilir. Sağlık uzmanınıza danışın."
                ]))

        if yas in ["65+"]:
            comments.append(random.choice([
                f"{yas} yaşlarda bazı ilaçlar potasyum seviyelerini düşürebilir. Düzenli potasyum takibi ve doktor kontrolü önemlidir.",
                f"{yas}'larda düşük potasyum, kalp ritmi sorunlarına neden olabilir. İlaçlarınızı doktorunuzla gözden geçirin.",
                f"Düşük potasyum seviyesi, {yas}'larda kas zayıflığı ve yorgunluğa yol açabilir. Beslenmenizi gözden geçirin.",
                "Yaş ilerledikçe potasyum dengesi bozulabilir. Bu durumu doktorunuzla paylaşın.",
                f"Bazı ilaçlar, {yas}'larda potasyum eksikliğine neden olabilir. Bir sağlık uzmanına danışın."
            ]))
        if "0 TL - 30.000 TL" in gelir:
            comments.append(random.choice([
                f"{gelir} düzeyine sahip kişilerde taze meyve ve sebze tüketimi kısıtlı olabilir, bu da potasyum eksikliğine yol açabilir. Beslenme danışmanlığı faydalı olabilir.",
                f"Potasyum açısından zengin gıdalara erişim kısıtlı olduğunda, {gelir} düzeyine sahip bireylerde eksiklik riski artar. Doktorunuza danışın.",
                f"Yetersiz beslenme, potasyum eksikliğine yol açabilir.{gelir} düzeyine sahipseniz bu duruma dikkat edin.",
                f"Potasyum eksikliği riski, {gelir} düzeyine sahip topluluklarda daha yüksek olabilir. Beslenme alışkanlıklarınızı gözden geçirin.",
                f"{gelir} düzeyindeki bireylerde potasyum eksikliği, beslenme yetersizliklerinden kaynaklanabilir. Tıbbi yardım alın."
            ]))
    else:
        comments.append(random.choice(HIGH_COMMENTS))
        if has_hereditary and "4. Böbrek Hastalığı (Kreatinin ile ilgili)" in kalitsal_hastalik_tip:
            comments.append(random.choice([
                "Ailenizde böbrek hastalığı öyküsü varsa, yüksek potasyum seviyesi böbrek fonksiyonları açısından değerlendirilmelidir.",
                "Kalıtsal yatkınlık durumunda yüksek potasyum değerleri, böbrek fonksiyonlarında bir sorun olduğunu düşündürebilir. Bir uzmana danışın.",
                "Akrabalarınızda böbrek hastalığı bulunması, yüksek potasyum seviyenizle ilişkili olabilir. Tıbbi geçmişinizi paylaşın.",
                "Yüksek potasyumun böbrek hastalıklarıyla genetik bağlantısı olabilir. Bu durumu doktorunuzla detaylıca konuşun.",
                "Genetik faktörler yüksek potasyumu etkiliyorsa, böbrek sağlığı için özel bir tedavi planı gerekebilir."
            ]))
    return " ".join(comments)

def get_ref_range(test_name):
    ranges = {
        'Hemoglobin': (12.6, 16.7),
        'Magnezyum': (1.6, 2.6),
        'Glukoz': (74, 106),
        'Üre': (0, 50),
        'Kreatinin': (0.67, 1.17),
        'D Vitamini': (30, 100),
        'Potasyum': (3.9, 5.1)
    }
    return ranges.get(test_name)

def is_in_range(value, ref_range):
    if ref_range is None or value is None:
        return False
    return ref_range[0] <= value <= ref_range[1]


COMMENT_COLORS = {
    'Hemoglobin': '#0a2a43',
    'Magnezyum': '#1b3b5b',
    'Glukoz': '#2c4c73',
    'Üre': '#3d5d8b',
    'Kreatinin': '#4e6ea3',
    'D Vitamini': '#5f7fbb',
    'Potasyum': '#7090d3'
}

def format_comment_box(param_name, comment):
    color = COMMENT_COLORS.get(param_name, '#556b82')
    return f"<div style='background-color: {color}; padding: 15px; margin-bottom: 10px; border-radius: 8px; box-shadow: 2px 2px 5px rgba(0,0,0,0.3);'><p style='color: white; font-weight: bold;'>{param_name}:</p><p style='color: white;'>{comment}</p></div>"

def compare_values(name, val1, val2, unit="", bolge=None, gelir=None, yas=None, has_hereditary=False, yak_derece=None, kalitsal_hastalik_tip=None):
    """
    İki laboratuvar değeri arasındaki farkı ve referans aralığına,
    ayrıca kullanıcı bilgilerine göre yorumlar.
    """
    if val1 is None and val2 is None:
        return f"Her iki tahlilde de bulunamadı."
    elif val1 is None:
        return f"İlk tahlilde bulunamadı. İkinci tahlilde {val2}{unit} olarak ölçüldü."
    elif val2 is None:
        return f"İkinci tahlilde bulunamadı. İlk tahlilde {val1}{unit} olarak ölçüldü."

    ref_range = get_ref_range(name)
    in_range1 = is_in_range(val1, ref_range)
    in_range2 = is_in_range(val2, ref_range)

    diff = val2 - val1

    significant_diff_threshold = 0.5
    is_significant = abs(diff) >= significant_diff_threshold

    comment_parts = []

    if not is_significant:
        if in_range1 and in_range2:
            comment_parts.append(random.choice([
                "Her iki tahlilde de normal aralıkta ve belirgin bir değişiklik yok. Bu durum olumlu.",
                "Değerler iki tahlilde de sağlıklı aralıkta seyrediyor, önemli bir fark bulunamadı.",
                "Normal sınırlarda stabil bir durum söz konusu. İyiye işaret.",
                "Sonuçlar tutarlı ve referans aralığında. Endişelenmeye gerek yok.",
                "İki ölçümde de normal ve sabit. Sağlığınız bu alanda stabil."
            ]))
        elif not in_range1 and not in_range2:
            comment_parts.append(random.choice([
                "Her iki tahlilde de referans aralığı dışında olmasına rağmen, belirgin bir değişiklik yok. Doktor kontrolü önerilir.",
                "İki ölçümde de referans dışı, ancak değişim küçük. Durumun takip edilmesi gerekiyor.",
                "Değerler hala referans aralığı dışında ve önemli bir iyileşme/kötüleşme göstermiyor. Bir uzmana danışın.",
                "Normalin dışında seyretmeye devam ediyor. Bu durumu doktorunuzla paylaşmalısınız.",
                "Her iki tahlilde de istenmeyen aralıkta. Belirgin bir değişiklik olmaması, sürekli takibin önemini vurgular."
            ]))
        elif in_range1 and not in_range2:
            comment_parts.append(random.choice([
                "İlk tahlilde normal aralıkta iken, ikinci tahlilde referans aralığının dışına çıkmış ancak değişim çok küçük. Takip edilmesi önerilir.",
                "Değer normalden hafifçe sapmış. Bu küçük değişimin nedenini takip etmek önemlidir.",
                "İkinci tahlilde normal aralığın dışına çıktı, ancak fark marginal. Bir sonraki kontrolde tekrar değerlendirilmesi iyi olur.",
                "Normalden küçük bir sapma var. Bu durumun nedenini doktorunuzla görüşün.",
                "İlk tahlil iyi olmasına rağmen, ikinci tahlilde hafif bir kötüleşme var. Bu küçük değişiklik takip edilmeli."
            ]))
        elif not in_range1 and in_range2:
            comment_parts.append(random.choice([
                "İlk tahlilde referans aralığı dışındayken, ikinci tahlilde normal aralığa dönmüş. Bu olumlu bir gelişme ancak değişim küçük.",
                "Olumlu bir gelişme! Değer normal aralığına geri döndü, ancak değişim hala küçük.",
                "İlk sonuç kötüydü, ancak şimdi normale dönmüş. Bu küçük iyileşme takip edilmeli.",
                "Değeriniz düzelme eğiliminde, bu iyi bir haber. Küçük bir değişim olsa da olumlu.",
                "Referans dışından normale doğru küçük bir adım atılmış. Bu gelişmeyi sürdürmek için doktorunuza danışın."
            ]))
    else:
        if diff > 0:
            comment_parts.append(f"Değerde önemli bir artış ({diff:.2f}{unit}) gözlemlendi. ")
            if in_range2:
                comment_parts.append(random.choice([
                    "Şu anda normal aralıkta olmasına rağmen, bu artışın nedeni araştırılmalıdır.",
                    "Normal sınırlarda kalmasına rağmen, bu belirgin artışın kaynağı için doktorunuza danışın.",
                    "Değeriniz yükseliyor olsa da hala kabul edilebilir aralıkta. Yine de dikkatli olun.",
                    "Normal aralıkta kalmasına karşın, önemli bir artış var. Bu durumu doktorunuzla paylaşın.",
                    "Bu ani artış, normal sınırlarda dahi olsa, bir uyarı işareti olabilir. Tıbbi yardım almayı düşünün."
                ]))
            else:
                comment_parts.append(random.choice([
                    "Değer şu anda referans aralığının üstünde. Tıbbi değerlendirme gereklidir.",
                    "Değeriniz referans aralığını aştı ve ciddi bir artış gösterdi. Hemen bir doktora başvurun.",
                    "Bu yüksek değer, acil müdahale gerektirebilir. Uzman bir hekimle iletişime geçin.",
                    "Değeriniz tehlikeli bir şekilde yüksek. Nedenini belirlemek için acil tıbbi testler yapılmalı.",
                    "Normalin çok üzerinde bir artış var. Bu durumun ciddi sonuçları olabilir, hemen doktora görünün."
                ]))
        else:
            comment_parts.append(f"Değerde önemli bir düşüş ({-diff:.2f}{unit}) gözlemlendi. ")
            if in_range2:
                comment_parts.append(random.choice([
                    "Şu anda normal aralıkta olmasına rağmen, bu düşüşün nedeni araştırılmalıdır.",
                    "Normal sınırlar içinde olmasına rağmen, bu belirgin düşüşün kaynağı için doktorunuza danışın.",
                    "Değeriniz düşüyor olsa da hala kabul edilebilir aralıkta. Yine de dikkatli olun.",
                    "Normal aralıkta kalmasına karşın, önemli bir düşüş var. Bu durumu doktorunuzla paylaşın.",
                    "Bu ani düşüş, normal sınırlarda dahi olsa, bir uyarı işareti olabilir. Tıbbi yardım almayı düşünün."
                ]))
            else:
                comment_parts.append(random.choice([
                    "Değer şu anda referans aralığının altında. Tıbbi değerlendirme gereklidir.",
                    "Alarm! Değeriniz referans aralığının altına düştü ve ciddi bir azalma gösterdi. Hemen bir doktora başvurun.",
                    "Bu düşük değer, acil müdahale gerektirebilir. Uzman bir hekimle iletişime geçin.",
                    "Değeriniz tehlikeli bir şekilde düşük. Nedenini belirlemek için acil tıbbi testler yapılmalı.",
                    "Normalin çok altında bir düşüş var. Bu durumun ciddi sonuçları olabilir, hemen doktora görünün."
                ]))


    if not in_range2 or is_significant:
        if name == "Hemoglobin":
            comment_parts.append(get_hemoglobin_comments(val2, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir))
        elif name == "Magnezyum":
            comment_parts.append(get_magnezyum_comments(val2, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir))
        elif name == "Glukoz":
            comment_parts.append(get_glukoz_comments(val2, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir))
        elif name == "Üre":
            comment_parts.append(get_ure_comments(val2, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir))
        elif name == "Kreatinin":
            comment_parts.append(get_kreatinin_comments(val2, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir))
        elif name == "D Vitamini":
            comment_parts.append(get_dvitamini_comments(val2, has_hereditary, yak_derece, kalitsal_hastalik_tip, gelir, bolge))
        elif name == "Potasyum":
            comment_parts.append(get_potasyum_comments(val2, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir))

    return " ".join(comment_parts)


def analyze_single_lab_results(pdf_path, bolge, gelir, yas, has_hereditary, yak_derece, kalitsal_hastalik_tip):
    if not pdf_path:
        return "Lütfen analiz için bir PDF dosyası yükleyin."

    text = extract_text_from_pdf(pdf_path)
    if "Hata:" in text:
        return text

    hemoglobin = extract_value(text, ["Hemoglobin"])
    magnezyum = extract_value(text, ["Magnezyum"])
    glukoz = extract_value(text, ["Glukoz", "Açlık Kan Şekeri"])
    ure = extract_value(text, ["Üre"])
    kreatinin = extract_value(text, ["Kreatinin"])
    d_vitamini = extract_value(text, ["D Vitamini"])
    potasyum = extract_value(text, ["Potasyum"])

    results_html = "<h3>Laboratuvar Sonuçlarınızın Yorumu:</h3>"
    results_html += format_comment_box("Hemoglobin", f"{hemoglobin if hemoglobin is not None else 'Bulunamadı'} - {get_hemoglobin_comments(hemoglobin, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir)}")
    results_html += format_comment_box("Magnezyum", f"{magnezyum if magnezyum is not None else 'Bulunamadı'} - {get_magnezyum_comments(magnezyum, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir)}")
    results_html += format_comment_box("Glukoz", f"{glukoz if glukoz is not None else 'Bulunamadı'} - {get_glukoz_comments(glukoz, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir)}")
    results_html += format_comment_box("Üre", f"{ure if ure is not None else 'Bulunamadı'} - {get_ure_comments(ure, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir)}")
    results_html += format_comment_box("Kreatinin", f"{kreatinin if kreatinin is not None else 'Bulunamadı'} - {get_kreatinin_comments(kreatinin, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir)}")
    results_html += format_comment_box("D Vitamini", f"{d_vitamini if d_vitamini is not None else 'Bulunamadı'} - {get_dvitamini_comments(d_vitamini, has_hereditary, yak_derece, kalitsal_hastalik_tip, gelir, bolge)}")
    results_html += format_comment_box("Potasyum", f"{potasyum if potasyum is not None else 'Bulunamadı'} - {get_potasyum_comments(potasyum, has_hereditary, yak_derece, kalitsal_hastalik_tip, yas, gelir)}")

    results_html += "<p style='font-size: 0.8em; color: #CCCCCC;'><b>Önemli Not:</b> Bu yorumlar genel bilgilendirme amaçlıdır. Kesin tanı ve tedavi için mutlaka bir sağlık uzmanına danışın.</p>"
    return results_html

def compare_lab_results(pdf_path1, pdf_path2, bolge, gelir, yas, has_hereditary, yak_derece, kalitsal_hastalik_tip):
    if not pdf_path1 or not pdf_path2:
        return "Lütfen karşılaştırma için iki PDF dosyası yükleyin."

    text1 = extract_text_from_pdf(pdf_path1)
    text2 = extract_text_from_pdf(pdf_path2)

    if "Hata:" in text1:
        return f"İlk PDF için hata: {text1}"
    if "Hata:" in text2:
        return f"İkinci PDF için hata: {text2}"

    hemoglobin1 = extract_value(text1, ["Hemoglobin"])
    magnezyum1 = extract_value(text1, ["Magnezyum"])
    glukoz1 = extract_value(text1, ["Glukoz", "Açlık Kan Şekeri"])
    ure1 = extract_value(text1, ["Üre"])
    kreatinin1 = extract_value(text1, ["Kreatinin"])
    d_vitamini1 = extract_value(text1, ["D Vitamini"])
    potasyum1 = extract_value(text1, ["Potasyum"])

    hemoglobin2 = extract_value(text2, ["Hemoglobin"])
    magnezyum2 = extract_value(text2, ["Magnezyum"])
    glukoz2 = extract_value(text2, ["Glukoz", "Açlık Kan Şekeri"])
    ure2 = extract_value(text2, ["Üre"])
    kreatinin2 = extract_value(text2, ["Kreatinin"])
    d_vitamini2 = extract_value(text2, ["D Vitamini"])
    potasyum2 = extract_value(text2, ["Potasyum"])

    comparison_html = "<p style='font-size: 0.8em; color: #CCCCCC;'><b>Önemli Not:</b> Bu karşılaştırmalar genel bilgilendirme amaçlıdır. Kesin tanı ve tedavi için mutlaka bir sağlık uzmanına danışın.</p>"

    comparison_html += "<h3>Laboratuvar Sonuçları Karşılaştırması:</h3>"
    comparison_html += "<table border='1'><tr><th>Parametre</th><th>İlk Tahlil</th><th>İkinci Tahlil</th><th>Değişim Yorumu</th></tr>"

    comparison_html += f"<tr><td>Hemoglobin</td><td>{hemoglobin1 if hemoglobin1 is not None else 'Yok'}</td><td>{hemoglobin2 if hemoglobin2 is not None else 'Yok'}</td><td>{compare_values('Hemoglobin', hemoglobin1, hemoglobin2, ' g/dL', bolge, gelir, yas, has_hereditary, yak_derece, kalitsal_hastalik_tip)}</td></tr>"
    comparison_html += f"<tr><td>Magnezyum</td><td>{magnezyum1 if magnezyum1 is not None else 'Yok'}</td><td>{magnezyum2 if magnezyum2 is not None else 'Yok'}</td><td>{compare_values('Magnezyum', magnezyum1, magnezyum2, ' mg/dL', bolge, gelir, yas, has_hereditary, yak_derece, kalitsal_hastalik_tip)}</td></tr>"
    comparison_html += f"<tr><td>Glukoz</td><td>{glukoz1 if glukoz1 is not None else 'Yok'}</td><td>{glukoz2 if glukoz2 is not None else 'Yok'}</td><td>{compare_values('Glukoz', glukoz1, glukoz2, ' mg/dL', bolge, gelir, yas, has_hereditary, yak_derece, kalitsal_hastalik_tip)}</td></tr>"
    comparison_html += f"<tr><td>Üre</td><td>{ure1 if ure1 is not None else 'Yok'}</td><td>{ure2 if ure2 is not None else 'Yok'}</td><td>{compare_values('Üre', ure1, ure2, ' mg/dL', bolge, gelir, yas, has_hereditary, yak_derece, kalitsal_hastalik_tip)}</td></tr>"
    comparison_html += f"<tr><td>Kreatinin</td><td>{kreatinin1 if kreatinin1 is not None else 'Yok'}</td><td>{kreatinin2 if kreatinin2 is not None else 'Yok'}</td><td>{compare_values('Kreatinin', kreatinin1, kreatinin2, ' mg/dL', bolge, gelir, yas, has_hereditary, yak_derece, kalitsal_hastalik_tip)}</td></tr>"
    comparison_html += f"<tr><td>D Vitamini</td><td>{d_vitamini1 if d_vitamini1 is not None else 'Yok'}</td><td>{d_vitamini2 if d_vitamini2 is not None else 'Yok'}</td><td>{compare_values('D Vitamini', d_vitamini1, d_vitamini2, ' ng/mL', bolge, gelir, yas, has_hereditary, yak_derece, kalitsal_hastalik_tip)}</td></tr>"
    comparison_html += f"<tr><td>Potasyum</td><td>{potasyum1 if potasyum1 is not None else 'Yok'}</td><td>{potasyum2 if potasyum2 is not None else 'Yok'}</td><td>{compare_values('Potasyum', potasyum1, potasyum2, ' mmol/L', bolge, gelir, yas, has_hereditary, yak_derece, kalitsal_hastalik_tip)}</td></tr>"

    comparison_html += "</table>"
    return comparison_html

custom_css = """
body {
    background-color: #1a2a3a;
}
.gradio-container {
    background-color: #2b3b4b;
    border-radius: 12px;
    box-shadow: 0 8px 16px rgba(0, 0, 0, 0.6);
    padding: 30px;
    color: #e0e6eb;
}
h1, h3 {
    color: #e0e6eb; /
    font-weight: bold;
}
p, label, .gr-form {
    color: #e0e6eb;
}
.gr-button {
    background-color: #3498db !important;
    color: white !important;
    border-radius: 8px;
    padding: 10px 20px;
    font-size: 1.1em;
    transition: background-color 0.3s ease;
}
.gr-button:hover {
    background-color: #2980b9 !important;
}
.gr-file-upload {
    border: 2px dashed #3498db;
    background-color: #3a4a5a;
    padding: 20px;
    border-radius: 10px;
    text-align: center;
    color: #e0e6eb;
}
table {
    width: 100%;
    border-collapse: collapse;
    margin-top: 25px;
    box-shadow: 0 4px 8px rgba(0, 0, 0, 0.4);
}
th, td {
    border: 1px solid #4a5a6a;
    padding: 12px;
    text-align: left;
    color: #e0e6eb;
}
th {
    background-color: #3a4a5a;
    font-weight: bold;
}

input[type='radio'] + label, input[type='checkbox'] + label {
    color: #e0e6eb;
}
.gr-radio-label {
    color: #e0e6eb !important;
}
.gr-checkbox-label {
    color: #e0e6eb !important;
}
"""

with gr.Blocks(theme=gr.themes.Base(), css=custom_css) as demo:
    gr.Markdown("# Laboratuvar Sonuçları Analizi ve Karşılaştırması 🧬")

    with gr.Tab("Tekli Rapor Analizi"):
        gr.Markdown("### Tek Bir Laboratuvar Raporunu Analiz Edin")
        with gr.Row():
            pdf_file_single = gr.File(label="Laboratuvar Sonucu PDF Dosyası Yükleyin", file_types=[".pdf"])
        with gr.Row():
            gr.Markdown("#### Kullanıcı Bilgileri")
        with gr.Row():
            bolge_input_single = gr.Radio(label="Yaşadığınız Bölge", choices=bolge_options, scale=1)
            gelir_input_single = gr.Radio(label="Gelir Durumunuz", choices=gelir_options, scale=1)
            yas_input_single = gr.Radio(label="Yaş Aralığınız", choices=yas_options, scale=1)
        with gr.Row():
            has_hereditary_input_single = gr.Checkbox(label="Ailenizde Kalıtsal Hastalık Öyküsü Var mı?")
        with gr.Row():
            yak_derece_input_single = gr.Radio(label="Kalıtsal Hastalığın Yakınlık Derecesi", choices=yak_derece_options, visible=False)
            kalitsal_hastalik_tip_input_single = gr.CheckboxGroup(label="Kalıtsal Hastalık Tipi(leri)", choices=kalitsal_hastalik_tip_options, visible=False)

        has_hereditary_input_single.change(
            lambda x: {
                yak_derece_input_single: gr.update(visible=x),
                kalitsal_hastalik_tip_input_single: gr.update(visible=x)
            },
            inputs=has_hereditary_input_single,
            outputs=[yak_derece_input_single, kalitsal_hastalik_tip_input_single]
        )

        analyze_button_single = gr.Button("Analiz Et")
        output_html_single = gr.HTML()

        analyze_button_single.click(
            analyze_single_lab_results,
            inputs=[
                pdf_file_single,
                bolge_input_single,
                gelir_input_single,
                yas_input_single,
                has_hereditary_input_single,
                yak_derece_input_single,
                kalitsal_hastalik_tip_input_single
            ],
            outputs=output_html_single
        )

    with gr.Tab("Rapor Karşılaştırması"):
        gr.Markdown("### İki Laboratuvar Raporunu Karşılaştırın")
        with gr.Row():
            pdf_file_compare1 = gr.File(label="İlk Laboratuvar Sonucu PDF Dosyası Yükleyin", file_types=[".pdf"])
            pdf_file_compare2 = gr.File(label="İkinci Laboratuvar Sonucu PDF Dosyası Yükleyin", file_types=[".pdf"])
        with gr.Row():
            gr.Markdown("#### Kullanıcı Bilgileri (Karşılaştırma İçin)")
        with gr.Row():
            bolge_input_compare = gr.Radio(label="Yaşadığınız Bölge", choices=bolge_options, scale=1)
            gelir_input_compare = gr.Radio(label="Gelir Durumunuz", choices=gelir_options, scale=1)
            yas_input_compare = gr.Radio(label="Yaş Aralığınız", choices=yas_options, scale=1)
        with gr.Row():
            has_hereditary_input_compare = gr.Checkbox(label="Ailenizde Kalıtsal Hastalık Öyküsü Var mı?")
        with gr.Row():
            yak_derece_input_compare = gr.Radio(label="Kalıtsal Hastalığın Yakınlık Derecesi", choices=yak_derece_options, visible=False)
            kalitsal_hastalik_tip_input_compare = gr.CheckboxGroup(label="Kalıtsal Hastalık Tipi(leri)", choices=kalitsal_hastalik_tip_options, visible=False)

        has_hereditary_input_compare.change(
            lambda x: {
                yak_derece_input_compare: gr.update(visible=x),
                kalitsal_hastalik_tip_input_compare: gr.update(visible=x)
            },
            inputs=has_hereditary_input_compare,
            outputs=[yak_derece_input_compare, kalitsal_hastalik_tip_input_compare]
        )

        compare_button = gr.Button("Karşılaştır")
        output_html_compare = gr.HTML()

        compare_button.click(
            compare_lab_results,
            inputs=[
                pdf_file_compare1,
                pdf_file_compare2,
                bolge_input_compare,
                gelir_input_compare,
                yas_input_compare,
                has_hereditary_input_compare,
                yak_derece_input_compare,
                kalitsal_hastalik_tip_input_compare
            ],
            outputs=output_html_compare
        )

demo.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b92c79d8880de32d51.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
